In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import geopandas as gpd
from shapely.geometry import Point
import fiona
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# df = pd.read_csv("https://data.cityofnewyork.us/api/views/pqfs-mqru/rows.csv?query=select+*+where+%60lpep_pickup_datetime%60+%3E%3D+%272016-01-01T00%3A00%3A00%27+AND+%60lpep_pickup_datetime%60+%3C+%272016-06-30T23%3A59%3A59%27&read_from_nbe=true&version=2.1&accessType=DOWNLOAD")

In [3]:
#2015 green taxi
#df = pd.read_csv('https://data.cityofnewyork.us/api/views/utt9-dvgj/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*+where+%60lpep_pickup_datetime%60+%3E%3D+%272015-01-01T00%3A00%3A00%27+AND+%60lpep_pickup_datetime%60+%3C+%272015-03-31T00%3A00%3A00%27')

In [4]:
#2014 green taxi
# df = pd.read_csv('https://data.cityofnewyork.us/api/views/7j25-xd5y/rows.csv?query=select+*+where+%60lpep_pickup_datetime%60+%3E%3D+%272014-10-01T00%3A00%3A00%27+AND+%60lpep_pickup_datetime%60+%3C+%272014-12-31T23%3A59%3A59%27&read_from_nbe=true&version=2.1&accessType=DOWNLOAD')

In [2]:
#2013 green taxi
df = pd.read_csv('https://data.cityofnewyork.us/api/views/h4pe-ymjc/rows.csv?accessType=DOWNLOAD')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210811 entries, 0 to 1210810
Data columns (total 20 columns):
pickup_datetime       1210811 non-null object
dropoff_datetime      1210811 non-null object
Store_and_fwd_flag    1210811 non-null object
rate_code             1210811 non-null int64
Dropoff_latitude      1210811 non-null float64
Passenger_count       1210811 non-null int64
Trip_distance         1210811 non-null float64
Fare_amount           1210811 non-null float64
Extra                 1210811 non-null float64
MTA_tax               1210811 non-null float64
Tip_amount            1210811 non-null float64
Tolls_amount          1210811 non-null float64
Ehail_fee             0 non-null float64
Total_amount          1210811 non-null float64
Payment_type          1210811 non-null int64
Trip_type             15211 non-null float64
Pickup_longitude      1210811 non-null float64
Pickup_latitude       1210811 non-null float64
Dropoff_longitude     1210811 non-null float64
vendor_id  

In [4]:
df.columns

Index(['pickup_datetime', 'dropoff_datetime', 'Store_and_fwd_flag',
       'rate_code', 'Dropoff_latitude', 'Passenger_count', 'Trip_distance',
       'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount',
       'Ehail_fee', 'Total_amount', 'Payment_type', 'Trip_type',
       'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude',
       'vendor_id'],
      dtype='object')

In [5]:
df = df[['pickup_datetime', 'Pickup_longitude', 
         'Pickup_latitude', 'Dropoff_longitude',
         'Dropoff_latitude', 'Passenger_count']]

In [6]:
# df = df[['lpep_pickup_datetime',
#        'PULocationID', 'DOLocationID',
#        'Passenger_count']]

In [7]:
df.shape

(1210811, 6)

In [8]:
df['month'] = df['pickup_datetime'].apply(lambda x: int(x[:2]))

In [9]:
df_pick = df[['month','Pickup_longitude','Pickup_latitude','Passenger_count']]
df_drop = df[['month','Dropoff_longitude','Dropoff_latitude','Passenger_count']]

In [10]:
df_pick.head()

,month,Pickup_longitude,Pickup_latitude,Passenger_count
0,9,0.000000,0.00000,1
1,11,-73.929573,40.75436,1
2,9,0.000000,0.00000,1
3,10,0.000000,0.00000,1
4,10,0.000000,0.00000,1


In [11]:
df_drop.head()

,month,Dropoff_longitude,Dropoff_latitude,Passenger_count
0,9,0.0,0.0,1
1,11,0.0,0.0,1
2,9,0.0,0.0,1
3,10,0.0,0.0,1
4,10,0.0,0.0,1


In [15]:
df = []

## Binning by zipcode

In [16]:
zipcode = gpd.read_file('ZIP_CODE/ZIP_CODE_040114.shp')
zipcode = zipcode.to_crs(epsg=4326)

In [17]:
zipcode = zipcode[['ZIPCODE','geometry']]

In [18]:
crs = {'init' :'epsg:4326'}

In [19]:
pick = [Point(xy) for xy in zip(df_pick.Pickup_longitude, df_pick.Pickup_latitude)]
drop = [Point(xy) for xy in zip(df_drop.Dropoff_longitude, df_drop.Dropoff_latitude)]

In [20]:
pick_gdf = gpd.GeoDataFrame(df_pick[['Passenger_count','month']], geometry=pick, crs=crs)
drop_gdf = gpd.GeoDataFrame(df_drop[['Passenger_count','month']], geometry=drop, crs=crs)

In [21]:
pick_gdf = gpd.sjoin(pick_gdf, zipcode, how='right', op='within').reset_index()

In [22]:
drop_gdf = gpd.sjoin(drop_gdf, zipcode, how='right', op='within').reset_index()

In [23]:
pick_gdf.head()

,index_right,index_left,Passenger_count,month,ZIPCODE,geometry
0,0,1165753.0,1.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
1,0,1031527.0,1.0,11.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
2,0,1042726.0,1.0,11.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
3,0,271146.0,2.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
4,0,513332.0,1.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...


In [24]:
drop_gdf.head()

,index_right,index_left,Passenger_count,month,ZIPCODE,geometry
0,0,937242.0,1.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
1,0,701387.0,1.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
2,0,304785.0,1.0,12.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
3,0,471002.0,1.0,10.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...
4,0,684191.0,1.0,11.0,11436,POLYGON ((-73.80584847647394 40.68290932644247...


In [25]:
del pick_gdf['index_right']
del pick_gdf['index_left']
del pick_gdf['geometry']

In [26]:
del drop_gdf['index_right']
del drop_gdf['index_left']
del drop_gdf['geometry']

In [27]:
pick_gdf = pick_gdf.fillna(0)
drop_gdf = drop_gdf.fillna(0)

## Groupby Month

In [28]:
# taxi17_pick = df.groupby(['month','PULocationID']).Passenger_count.sum().unstack().T.reset_index()
# taxi17_drop = df.groupby(['month','DOLocationID']).Passenger_count.sum().unstack().T.reset_index()

In [29]:
# taxi17_pick.columns = ['PULocationID', 1607, 1608, 1609, 1610, 1611, 1612]
# taxi17_drop.columns = ['DOLocationID', 1607, 1608, 1609, 1610, 1611, 1612]

In [30]:
# taxi17_pick.to_csv('taxi_data/taxi16_pick_monthly2.csv')
# taxi17_drop.to_csv('taxi_data/taxi16_drop_monthly2.csv')

In [31]:
taxi_pick = pick_gdf.groupby(['month','ZIPCODE']).Passenger_count.sum().unstack().T.reset_index().fillna(0)
taxi_drop = drop_gdf.groupby(['month','ZIPCODE']).Passenger_count.sum().unstack().T.reset_index().fillna(0)

In [32]:
del taxi_pick[0]
del taxi_drop[0]

In [33]:
taxi_pick.head()

month,ZIPCODE,8.0,9.0,10.0,11.0,12.0
0,00083,8.0,61.0,185.0,280.0,920.0
1,10001,0.0,1.0,5.0,9.0,10.0
2,10002,6.0,7.0,13.0,18.0,4.0
3,10003,0.0,0.0,0.0,10.0,20.0
4,10004,0.0,3.0,2.0,1.0,1.0


In [34]:
taxi_drop.head()

month,ZIPCODE,8.0,9.0,10.0,11.0,12.0
0,00083,62.0,370.0,1418.0,3442.0,4480.0
1,10001,207.0,1061.0,2814.0,6441.0,8982.0
2,10002,140.0,892.0,1967.0,4290.0,6312.0
3,10003,98.0,759.0,2168.0,4509.0,6632.0
4,10004,27.0,75.0,274.0,476.0,759.0


In [35]:
taxi_pick.columns = ['ZIPCODE', 1308, 1309, 1310,1311,1312]
taxi_drop.columns = ['ZIPCODE', 1308, 1309, 1310,1311,1312]
# taxi_pick.columns = ['ZIPCODE', 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412]
# taxi_drop.columns = ['ZIPCODE', 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410, 1411, 1412]

In [36]:
taxi_pick.to_csv('taxi_data/taxi13_pick_monthly.csv')
taxi_drop.to_csv('taxi_data/taxi13_drop_monthly.csv')

In [40]:
df1_pick = pd.read_csv('taxi_data/taxi13_pick_monthly.csv')
df1_drop = pd.read_csv('taxi_data/taxi13_drop_monthly.csv')
df1_pick.head()

,Unnamed: 0,ZIPCODE,1308,1309,1310,1311,1312
0,0,83,8.0,61.0,185.0,280.0,920.0
1,1,10001,0.0,1.0,5.0,9.0,10.0
2,2,10002,6.0,7.0,13.0,18.0,4.0
3,3,10003,0.0,0.0,0.0,10.0,20.0
4,4,10004,0.0,3.0,2.0,1.0,1.0


In [41]:
df1_drop.head()

,Unnamed: 0,ZIPCODE,1308,1309,1310,1311,1312
0,0,83,62.0,370.0,1418.0,3442.0,4480.0
1,1,10001,207.0,1061.0,2814.0,6441.0,8982.0
2,2,10002,140.0,892.0,1967.0,4290.0,6312.0
3,3,10003,98.0,759.0,2168.0,4509.0,6632.0
4,4,10004,27.0,75.0,274.0,476.0,759.0


In [37]:
# # Read the quarterly data
# df1_pick = pd.read_csv('taxi_data/taxi14_pick_1.csv')
# df1_drop = pd.read_csv('taxi_data/taxi14_drop_1.csv')
# df2_pick = pd.read_csv('taxi_data/taxi14_pick_2.csv')
# df2_drop = pd.read_csv('taxi_data/taxi14_drop_2.csv')
# df3_pick = pd.read_csv('taxi_data/taxi14_pick_3.csv')
# df3_drop = pd.read_csv('taxi_data/taxi14_drop_3.csv')
# df4_pick = pd.read_csv('taxi_data/taxi14_pick_4.csv')
# df4_drop = pd.read_csv('taxi_data/taxi14_drop_4.csv')

# del df1_pick['Unnamed: 0']
# del df1_drop['Unnamed: 0']
# del df2_pick['Unnamed: 0']
# del df2_drop['Unnamed: 0']
# del df3_pick['Unnamed: 0']
# del df3_drop['Unnamed: 0']
# del df4_pick['Unnamed: 0']
# del df4_drop['Unnamed: 0']

In [38]:
# #Merge the quarterly data together
# df_pick = pd.merge(df1_pick,df2_pick,on='ZIPCODE')
# df_pick = pd.merge(df_pick,df3_pick,on='ZIPCODE')
# df_pick = pd.merge(df_pick,df4_pick,on='ZIPCODE')

# df_drop = pd.merge(df1_drop,df2_drop,on='ZIPCODE')
# df_drop = pd.merge(df_drop,df3_drop,on='ZIPCODE')
# df_drop = pd.merge(df_drop,df4_drop,on='ZIPCODE')

In [42]:
# # Read the monthly data
# df1_pick = pd.read_csv('taxi_data/taxi13_pick_monthly.csv')
# df1_drop = pd.read_csv('taxi_data/taxi13_drop_monthly.csv')
# df2_pick = pd.read_csv('taxi_data/taxi14_pick_monthly.csv')
# df2_drop = pd.read_csv('taxi_data/taxi14_drop_monthly.csv')
# df3_pick = pd.read_csv('taxi_data/taxi15_pick_monthly.csv')
# df3_drop = pd.read_csv('taxi_data/taxi15_drop_monthly.csv')
# df4_pick = pd.read_csv('taxi_data/taxi16_pick_monthly1.csv')
# df4_drop = pd.read_csv('taxi_data/taxi16_drop_monthly1.csv')

# del df1_pick['Unnamed: 0']
# del df1_drop['Unnamed: 0']
# del df2_pick['Unnamed: 0']
# del df2_drop['Unnamed: 0']
# del df3_pick['Unnamed: 0']
# del df3_drop['Unnamed: 0']
# del df4_pick['Unnamed: 0']
# del df4_drop['Unnamed: 0']

In [43]:
# #Merge the monthly data together
# df_pick = pd.merge(df1_pick,df2_pick,on='ZIPCODE')
# df_pick = pd.merge(df_pick,df3_pick,on='ZIPCODE')
# df_pick = pd.merge(df_pick,df4_pick,on='ZIPCODE')

# df_drop = pd.merge(df1_drop,df2_drop,on='ZIPCODE')
# df_drop = pd.merge(df_drop,df3_drop,on='ZIPCODE')
# df_drop = pd.merge(df_drop,df4_drop,on='ZIPCODE')

In [54]:
df1_drop.head()

,PULocationID,1607,1608,1609,1610,1611,1612
0,1,1022.0,778.0,639.0,746.0,704.0,791.0
1,2,1.0,NaN,NaN,NaN,6.0,NaN
2,3,694.0,612.0,607.0,595.0,550.0,563.0
3,4,2900.0,2710.0,2481.0,2600.0,2295.0,2338.0
4,5,9.0,3.0,9.0,8.0,11.0,11.0


In [65]:
#merge monthly data(16-17)
df1_pick = pd.read_csv('taxi_data/taxi16_pick_monthly2.csv')
df1_drop = pd.read_csv('taxi_data/taxi16_drop_monthly2.csv')
df2_pick = pd.read_csv('taxi_data/taxi17_pick_monthly.csv')
df2_drop = pd.read_csv('taxi_data/taxi17_drop_monthly.csv')

del df1_pick['Unnamed: 0']
del df1_drop['Unnamed: 0']
del df2_pick['Unnamed: 0']
del df2_drop['Unnamed: 0']

df_pick = pd.merge(df1_pick,df2_pick,on='LocationID')
df_drop = pd.merge(df1_drop,df2_drop,on='LocationID')

In [68]:
df_pick.head()

,LocationID,1607,1608,1609,1610,1611,1612,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712
0,1,57.0,47.0,35.0,40.0,21.0,34.0,17.0,17.0,32.0,26.0,41.0,25.0,39.0,23.0,20.0,62.0,34.0,17.0
1,3,241.0,224.0,195.0,197.0,152.0,181.0,172.0,192.0,244.0,206.0,224.0,270.0,348.0,381.0,402.0,467.0,499.0,543.0
2,4,NaN,2.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,10.0,3.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN
3,5,NaN,NaN,3.0,1.0,1.0,7.0,NaN,1.0,6.0,NaN,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN
4,6,17.0,6.0,11.0,20.0,22.0,22.0,15.0,9.0,9.0,13.0,10.0,17.0,8.0,13.0,14.0,42.0,6.0,3.0


In [69]:
df_drop.head()

,LocationID,1607,1608,1609,1610,1611,1612,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712
0,1,1022.0,778.0,639.0,746.0,704.0,791.0,436.0,475.0,577.0,607.0,645.0,522.0,522.0,457.0,501.0,528.0,495.0,478.0
1,2,1.0,NaN,NaN,NaN,6.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,7.0,10.0,6.0,2.0,1.0,NaN
2,3,694.0,612.0,607.0,595.0,550.0,563.0,436.0,496.0,629.0,483.0,511.0,559.0,553.0,614.0,636.0,614.0,672.0,736.0
3,4,2900.0,2710.0,2481.0,2600.0,2295.0,2338.0,1980.0,2063.0,2024.0,2064.0,2003.0,1727.0,1779.0,1581.0,1603.0,1783.0,1504.0,1306.0
4,5,9.0,3.0,9.0,8.0,11.0,11.0,NaN,5.0,8.0,10.0,13.0,11.0,8.0,3.0,13.0,9.0,6.0,2.0


In [70]:
df_pick.to_csv('taxi_data/green_taxi_pick_monthly(16-17).csv')
df_drop.to_csv('taxi_data/green_taxi_drop_monthly(16-17).csv')

In [74]:
# test1 = pd.read_csv('taxi_data/green_taxi_pick_monthly(16-17).csv')
# test2 = pd.read_csv('taxi_data/green_taxi_pick_monthly(16-17).csv')
# test1

In [73]:
#test2